<a href="https://colab.research.google.com/github/alexbrun01/Book-Data-Science-do-Zero-/blob/main/Sistema_de_Previs%C3%A3o_de_Pre%C3%A7os_de_Ativos_Financeiros_com_Aprendizado_de_M%C3%A1quina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.0 - Importando Bibliotecas 

In [65]:
import os
import shutil
import pandas as pd
import yfinance as yf
from prophet import Prophet
import plotly.express as px

In [66]:
!pip install pandas yfinance matplotlib plotly==4.14.3 fbprophet scikit-learn ta
import ta
from prophet import Prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet
  Running setup.py clean for fbprophet
Failed to build fbprophet
  error: subprocess-exited-with-error
  
  × Running setup.py install for fbprophet did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for fbprophet ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> fbprophet

note: This is an issue with the package mentioned above, not pip.
hint: See

# 2.0 Install additional libraries

In [67]:

!pip install pandas yfinance matplotlib plotly==4.14.3 fbprophet scikit-learn ta
!pip install prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet
  Running setup.py clean for fbprophet
Failed to build fbprophet
  error: subprocess-exited-with-error
  
  × Running setup.py install for fbprophet did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for fbprophet ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> fbprophet

note: This is an issue with the package mentioned above, not pip.
hint: See

#3.0 Check if "prophet" directory exists and remove it if it does

In [69]:
if os.path.isdir('prophet'):
    print('Removendo diretório existente...')
    shutil.rmtree('prophet')

#4.0 Specify stock ticker of interest

In [70]:
symbol = 'BBDC4.SA'
start_date = '2015-01-01'
end_date = '2023-04-19'

#5.0 Download historical stock data for ticker from 2010-01-01 to 2023-04-17

In [71]:
print('Baixando dados...')
data = yf.download(symbol, start=start_date, end=end_date)

Baixando dados...
[*********************100%***********************]  1 of 1 completed


#6.0 Rename columns in training dataset

In [72]:
data = data.rename({'Adj Close': 'y'}, axis=1)


# 7.0 Verificação de valores 

In [73]:
if data.isnull().values.any():
    data = data.fillna(method='ffill')

#8.0 Conversão de coluna para data/hora

In [85]:
# 8.0 Converter a coluna de data/hora para formato de data/hora, se necessário
if 'Date' in data.columns:
    if not isinstance(data['ds'].dtype, pd.DatetimeTZDtype):
        data['ds'] = pd.to_datetime(data['ds'])
    data['Date'] = pd.to_datetime(data['Date'])
else:
    print("Column 'Date' does not exist in the DataFrame")

Column 'Date' does not exist in the DataFrame


In [87]:
# Codigo Open
if 'ds' in data.columns:
    if not isinstance(data['ds'].dtype, pd.DatetimeTZDtype):
        data['ds'] = pd.to_datetime(data['ds'])
else:
    print("Column 'ds' does not exist in the DataFrame")
    
if 'Date' in data.columns:
    if not isinstance(data['Date'].dtype, pd.DatetimeTZDtype):
        data['Date'] = pd.to_datetime(data['Date'])
else:
    print("Column 'Date' does not exist in the DataFrame")


Column 'ds' does not exist in the DataFrame
Column 'Date' does not exist in the DataFrame


Este código ( Open ) verifica se as colunas 'ds' e 'Date' existem no DataFrame e se seus tipos são de fato do tipo de data/hora. Caso não sejam, ele converte a coluna para o tipo correto. Se a coluna não existir, uma mensagem de erro é exibida.



*texto em itálico*

#9.0 Normalizar os dados


In [95]:
#Ordenar os dados pelo horário da coluna de data/hora
if 'ds' in data.columns:
    data = data.sort_values(by='ds')
elif 'Date' in data.columns:
    data = data.sort_values(by='Date')
else:
    # Nenhuma das colunas 'ds' ou 'Date' existe no DataFrame
    pass


In [94]:
# Normalizar os dados, se necessário
if 'y' in data.columns:
    scaler = MinMaxScaler()
    data['y'] = scaler.fit_transform(data[['y']])


#10.0 Criar novas features para uso no modelo, como médias móveis, indicadores técnicos, etc.

In [97]:
if 'y' in data.columns:
    data['ma7'] = data['y'].rolling(window=7).mean()
    data['ma14'] = data['y'].rolling(window=14).mean()
    data = data.dropna()

# 11.0 Inicializar o modelo Prophet e treiná-lo com os dados pré-processados

#Treiando o modelo 

O Prophet requer que os dados sejam fornecidos em um DataFrame com as seguintes colunas:

ds: data/hora da observação
y: valor da observação
Temos que ter certeza que as colunas estão em formato correto antes de continuar. Podemos fazer isso com o seguinte código:

Este código verifica se a coluna 'y' existe no DataFrame e, se existir, calcula as médias móveis de 7 e 14 dias para a coluna 'y' e adiciona as novas colunas 'Média Móvel de 7 dias' e 'média movel de 14 dias' ao DataFrame. Em seguida, remove as linhas que contêm valores ausentes resultantes do cálculo das médias móveis usando o método dropna().

A média móvel é uma técnica estatística que suaviza a série temporal, calculando a média dos valores em um período de tempo específico. No código em questão, a média móvel é calculada para a coluna 'y' do DataFrame 'data' em janelas de 7 e 14 dias, respectivamente, e os resultados são armazenados nas novas colunas 'ma7' e 'ma14'. Essas novas colunas podem ser usadas como recursos (features) para treinar um modelo de previsão, como o Prophet.

In [109]:
if 'ds' in data.columns and 'y' in data.columns:
    model = Prophet()
    model.fit(data[['ds', 'y']])

    # Criar DataFrame com datas futuras para previsão
    future = model.make_future_dataframe(periods=365)

    # Fazer previsões para as datas futuras
    forecast = model.predict(future)


#12.0 Realizando previsões com o modelo treinado


    # Criar DataFrame com datas futuras para previsão
    future = model.make_future_dataframe(periods=365)- 
    
    - No exemplo acima, estamos fazendo a previsão para 365 dias (1 ano) no futuro. O parâmetro periods do método "make_future_dataframe" indica quantos períodos (dias, semanas, meses etc.) 

    # Fazer previsões para as datas futuras
    forecast = model.predict(future)

    - O resultado da previsão será armazenado na variável forecast



Para isso, é necessário criar um DataFrame com as datas futuras para as quais se deseja fazer a previsão e, em seguida, chamar o método predict do modelo, passando esse DataFrame como argumento. O código ficaria da seguinte forma:

#13.0 Compute exponential moving average with a span of 21 days for "y" column in entire dataset

In [43]:
# Make predictions for future dates
forecast = model.predict(pd.merge(future_dates[['ds', 'weekday']], train, on='ds'))

ValueError: ignored

In [42]:
fig = px.line(pd.concat([train, test]), x='ds', y='y', title=f"{ticker} Stock Price")
fig.add_scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted')
fig.show()

ModuleNotFoundError: ignored